<a href="https://colab.research.google.com/github/ThanaVi/THCovid19_Report/blob/master/ETL_THProvince_Lat_Long.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Target 
เก็บข้อมูล Latitude เเละ Longtitude ของจังหวัดในประเทศไทยในรูปแบบ "Latitude,Longtitude" สำหรับใช้ใน Geo Map ของ Google Data studio




#Result

ข้อมูลจาก Source

![alt text](https://drive.google.com/uc?id=1qbFclBv0QOGFOdoABgDEsqpKCWaH75qZ)

ข้อมูลที่ได้จาก Transformation

![alt text](https://drive.google.com/uc?id=1t3goOwsWC7OAZd4ck8CSkP23I7rqKIZZ)

#Install Pyspark

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xzvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

!pip install pyspark

# Import library

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.sql.types import StringType

# Create spark session

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

# Extract



ไฟล์ข้อมูลพิกัด LAT/LONG ที่ตั้งตำบล

Source: https://data.go.th/dataset/item_c6d42e1b-3219-47e1-b6b7-dfe914f27910


In [0]:
tambon_df = spark.read.csv('/content/drive/My Drive/Project/Covid19/tambon_lat_long_data.csv', header=True, inferSchema=True)
tambon_df

DataFrame[AD_LEVEL: int, TA_ID: int, TAMBON_T: string, TAMBON_E: string, AM_ID: int, AMPHOE_T: string, AMPHOE_E: string, CH_ID: int, CHANGWAT_T: string, CHANGWAT_E: string, LAT: double, LONG: double]

In [0]:
tambon_df.show()

+--------+------+--------------+----------+-----+-----------------+--------------------+-----+---------------+-------------+------+-------+
|AD_LEVEL| TA_ID|      TAMBON_T|  TAMBON_E|AM_ID|         AMPHOE_T|            AMPHOE_E|CH_ID|     CHANGWAT_T|   CHANGWAT_E|   LAT|   LONG|
+--------+------+--------------+----------+-----+-----------------+--------------------+-----+---------------+-------------+------+-------+
|       4|910106|ต. เกาะสาหร่าย|  Ko Sarai| 9101|             null|        Mueang Satun|   91|        จ. สตูล|        Satun| 6.546| 99.706|
|       4|210114|   ต. มาบตาพุด| Maptaphut| 2101|    อ. เมืองระยอง|       Mueang Rayong|   21|       จ. ระยอง|       Rayong|12.646|101.171|
|       4|210114|   ต. มาบตาพุด| Maptaphut| 2101|    อ. เมืองระยอง|       Mueang Rayong|   21|       จ. ระยอง|       Rayong|12.645| 101.17|
|       4|210114|   ต. มาบตาพุด| Maptaphut| 2101|    อ. เมืองระยอง|       Mueang Rayong|   21|       จ. ระยอง|       Rayong|12.649|101.174|
|       4|210114|   

# Transform



จากข้อมูล 
เนื่องจากในหนึ่งจังหวัดมี LAT/LONG อยู่หลายค่าตามตำบลเเละมีค่าซ้ำบ้าง จึงทำการเลือก LAT/LONG เฉพาะอำเภอเมือง

เลือกข้อมูลที่เป็นอำเภอเมือง โดยใช้คอลัมน์ AMPHOE_E เทียบกับ CHANGWAT_E ว่ามีชื่อเดียวกันไหม 

In [0]:
province_df = tambon_df.select('CH_ID', 'AMPHOE_E', 'CHANGWAT_E', 'CHANGWAT_T', 'LAT', 'LONG')\
                      .where('AMPHOE_E rlike CHANGWAT_E')
province_df.show()

+-----+--------------------+-------------+------------+------+-------+
|CH_ID|            AMPHOE_E|   CHANGWAT_E|  CHANGWAT_T|   LAT|   LONG|
+-----+--------------------+-------------+------------+------+-------+
|   91|        Mueang Satun|        Satun|     จ. สตูล| 6.546| 99.706|
|   21|       Mueang Rayong|       Rayong|    จ. ระยอง|12.646|101.171|
|   21|       Mueang Rayong|       Rayong|    จ. ระยอง|12.645| 101.17|
|   21|       Mueang Rayong|       Rayong|    จ. ระยอง|12.649|101.174|
|   21|       Mueang Rayong|       Rayong|    จ. ระยอง|12.644|101.169|
|   86|     Mueang Chumphon|     Chumphon|    จ. ชุมพร|10.459| 99.403|
|   44|Mueang Maha Sarakham|Maha Sarakham|จ. มหาสารคาม|16.162|103.484|
|   86|     Mueang Chumphon|     Chumphon|    จ. ชุมพร|10.477| 99.369|
|   91|        Mueang Satun|        Satun|     จ. สตูล| 6.672| 99.844|
|   86|     Mueang Chumphon|     Chumphon|    จ. ชุมพร|10.466| 99.359|
|   91|        Mueang Satun|        Satun|     จ. สตูล| 6.705| 99.799|
|   91

เมื่อได้อำเภอเมืองเเล้ว จะเห็นว่า LAT/LONG ยังมีหลายค่าอยู่ จึงนำมาเฉลี่ย 

In [0]:
province_avg_latlong_df = province_df.groupBy('CH_ID', 'CHANGWAT_E', 'CHANGWAT_T')\
                                      .agg(fn.mean('LAT').alias('AVG_LAT'),fn.mean('LONG').alias('AVG_LONG'))
province_avg_latlong_df.show()                          

+-----+----------------+--------------+------------------+------------------+
|CH_ID|      CHANGWAT_E|    CHANGWAT_T|           AVG_LAT|          AVG_LONG|
+-----+----------------+--------------+------------------+------------------+
|   91|           Satun|       จ. สตูล| 6.617406249999999|      99.669921875|
|   31|        Buri Ram|  จ. บุรีรัมย์|14.959684210526314|103.06421052631579|
|   39|Nong Bua Lam Phu|จ. หนองบัวลำภู|17.167933333333334|102.39466666666668|
|   15|       Ang Thong|    จ. อ่างทอง|14.582999999999998|100.44899999999998|
|   82|       Phang-nga|      จ. พังงา| 8.469222222222223|            98.514|
|   38|       Bueng Kan|     จ. บึงกาฬ|18.318416666666668|103.58116666666668|
|   96|      Narathiwat|   จ. นราธิวาส|6.3878571428571425|101.81457142857143|
|   75| Samut Songkhram|จ. สมุทรสงคราม|13.407181818181819| 99.99809090909093|
|   93|     Phatthalung|     จ. พัทลุง| 7.606733333333334|100.07346666666668|
|   57|      Chiang Rai|   จ. เชียงราย|           19.8986|      

จะได้ LAT/LONG ของแต่ละจังหวัดมา เเล้วตรวจว่าครบ 77 จังหวัดหรือไม่

In [0]:
province_avg_latlong_df.count()

74

พบว่าขาดไป 3 จังหวัด ตรวจว่าจังหวัดไหนหายไป

##Problem Solving

สร้าง list ของจังหวัดทั้งหมดที่มีในตาราง amphoe_df

In [0]:
all_provinces = tambon_df.select('CHANGWAT_E').distinct().collect()
len(all_provinces)

78

In [0]:
all_provinces[:5]

[Row(CHANGWAT_E='Chachoengsao'),
 Row(CHANGWAT_E='Phra Nakhon Si Ayutthaya'),
 Row(CHANGWAT_E='Phitsanulok'),
 Row(CHANGWAT_E='Sakon Nakhon'),
 Row(CHANGWAT_E='Ratchaburi')]

In [0]:
all_provinces[2][0]

'Phitsanulok'

In [0]:
list_provinces = [all_provinces[i][0] for i in range(len(all_provinces))]

ได้ list ของจังหวัดทั้งหมดที่มี

In [0]:
list_provinces[:10]

['Chachoengsao',
 'Phra Nakhon Si Ayutthaya',
 'Phitsanulok',
 'Sakon Nakhon',
 'Ratchaburi',
 'Lampang',
 'Chiang Mai',
 'Narathiwat',
 'Phetchabun',
 'Uthai Thani']

สร้าง list จังหวัดจาก province_avg_latlong_df

In [0]:
now_provinces = province_avg_latlong_df.select('CHANGWAT_E').collect()
len(now_provinces)

74

In [0]:
now_provinces[:5]

[Row(CHANGWAT_E='Satun'),
 Row(CHANGWAT_E='Buri Ram'),
 Row(CHANGWAT_E='Nong Bua Lam Phu'),
 Row(CHANGWAT_E='Ang Thong'),
 Row(CHANGWAT_E='Phang-nga')]

In [0]:
list_now_provinces = [now_provinces[i][0] for i in range(len(now_provinces))]

In [0]:
list_now_provinces[:10]

['Satun',
 'Buri Ram',
 'Nong Bua Lam Phu',
 'Ang Thong',
 'Phang-nga',
 'Bueng Kan',
 'Narathiwat',
 'Samut Songkhram',
 'Phatthalung',
 'Chiang Rai']

ได้ list ของจังหวัดที่มีอยู่

ตรวจสอบจังหวัดที่หายไป

In [0]:
lost_provinces = []
for province in list_provinces:
  if province not in  list_now_provinces:
    lost_provinces.append(province)

In [0]:
print(lost_provinces)

[None, 'Samut Prakarn', 'Sa kaeo', 'Suphan Buri']


ได้จังหวัดที่หายไป 3 จังหวัด

ตรวจสอบข้อมูลแต่ละจังหวัดที่หายไปจาก tambon_df

In [0]:
tambon_df.select('AMPHOE_E', 'CHANGWAT_E').where(tambon_df['CHANGWAT_E'] == lost_provinces[1]).show()

+-------------------+-------------+
|           AMPHOE_E|   CHANGWAT_E|
+-------------------+-------------+
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|Mueang Samut Prakan|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|Mueang Samut Prakan|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|Mueang Samut Prakan|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|      Phra Pradaeng|Samut Prakarn|
|Mueang Samut Prakan|Samut Prakarn|
|   Phra Samut Chedi|Samut Prakarn|
+-------------------+-------------+
only showing top 20 rows



In [0]:
tambon_df.select('AMPHOE_E', 'CHANGWAT_E').where(tambon_df['CHANGWAT_E'] == lost_provinces[2]).show()

+---------------+----------+
|       AMPHOE_E|CHANGWAT_E|
+---------------+----------+
|  Aranyaprathet|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
|    Khao Chakan|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
|      Khok Sung|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
| Mueang Sa Kaeo|   Sa kaeo|
|Watthana Nakhon|   Sa kaeo|
|     Khlong Hat|   Sa kaeo|
|     Khlong Hat|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
|     Khlong Hat|   Sa kaeo|
|     Khlong Hat|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
|     Khlong Hat|   Sa kaeo|
|Watthana Nakhon|   Sa kaeo|
|  Aranyaprathet|   Sa kaeo|
+---------------+----------+
only showing top 20 rows



In [0]:
tambon_df.select('AMPHOE_E', 'CHANGWAT_E').where(tambon_df['CHANGWAT_E'] == lost_provinces[3]).show()

+-------------------+-----------+
|           AMPHOE_E| CHANGWAT_E|
+-------------------+-----------+
|Doem Bang Nang Buat|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
|        Bang Pla Ma|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
|      Song Phi Nong|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
|      Song Phi Nong|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
|            U Thong|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
|         Si Prachan|Suphan Buri|
| Mueang Saphan Buri|Suphan Buri|
|         Si Prachan|Suphan Buri|
|            U Thong|Suphan Buri|
|            U Thong|Suphan Buri|
|            U Thong|Suphan Buri|
+-------------------+-----------+
only showing top 20 rows



สร้าง dataframe ที่เก็บข้อมูลจังหวัดที่หายไป

In [0]:
lost_df = tambon_df.select('CH_ID', 'AMPHOE_E', 'CHANGWAT_E', 'CHANGWAT_T', 'LAT', 'LONG')\
                    .where(((tambon_df['CHANGWAT_E'] == lost_provinces[1]) | 
                            (tambon_df['CHANGWAT_E'] == lost_provinces[2]) | 
                            (tambon_df['CHANGWAT_E'] == lost_provinces[3])) &
                            (tambon_df['AMPHOE_E'].rlike('Mueang')))
lost_df.show()              

+-----+-------------------+-------------+--------------+------+-------+
|CH_ID|           AMPHOE_E|   CHANGWAT_E|    CHANGWAT_T|   LAT|   LONG|
+-----+-------------------+-------------+--------------+------+-------+
|   11|Mueang Samut Prakan|Samut Prakarn|จ. สมุทรปราการ|13.623|100.565|
|   11|Mueang Samut Prakan|Samut Prakarn|จ. สมุทรปราการ|13.622|100.581|
|   11|Mueang Samut Prakan|Samut Prakarn|จ. สมุทรปราการ|13.622|100.605|
|   72| Mueang Saphan Buri|  Suphan Buri| จ. สุพรรณบุรี|14.471|100.116|
|   11|Mueang Samut Prakan|Samut Prakarn|จ. สมุทรปราการ|13.593|100.597|
|   72| Mueang Saphan Buri|  Suphan Buri| จ. สุพรรณบุรี|14.434|100.108|
|   11|Mueang Samut Prakan|Samut Prakarn|จ. สมุทรปราการ|13.634| 100.61|
|   11|Mueang Samut Prakan|Samut Prakarn|จ. สมุทรปราการ|13.572|100.621|
|   72| Mueang Saphan Buri|  Suphan Buri| จ. สุพรรณบุรี|14.506|100.107|
|   11|Mueang Samut Prakan|Samut Prakarn|จ. สมุทรปราการ|13.649|100.617|
|   72| Mueang Saphan Buri|  Suphan Buri| จ. สุพรรณบุรี| 14.47|1

เฉลี่ย LAT/LONG จังหวัดที่หายไป

In [0]:
lost_avg_latlong_df = lost_df.groupBy('CH_ID', 'CHANGWAT_E', 'CHANGWAT_T')\
                              .agg(fn.mean('LAT').alias('AVG_LAT'),fn.mean('LONG').alias('AVG_LONG'))
lost_avg_latlong_df.show()

+-----+-------------+--------------+---------+------------------+
|CH_ID|   CHANGWAT_E|    CHANGWAT_T|  AVG_LAT|          AVG_LONG|
+-----+-------------+--------------+---------+------------------+
|   72|  Suphan Buri| จ. สุพรรณบุรี|  14.4804|100.09329999999999|
|   27|      Sa kaeo|    จ. สระแก้ว|13.874125|           102.073|
|   11|Samut Prakarn|จ. สมุทรปราการ|   13.587|100.62938461538464|
+-----+-------------+--------------+---------+------------------+



นำ dataframe ที่เก็บข้อมูลที่หายไปมารวมกับ dataframe ข้อมูลที่มีอยู่

In [0]:
province_latlong_df = province_avg_latlong_df.unionByName(lost_avg_latlong_df)
province_latlong_df.count()

77

ตรวจสอบข้อมูล ได้ครบ 77 จังหวัด

สร้างคอลัมน์ LAT_LONG เพื่อเก็บ LAT/LONG ในรูปแปป LAT,LONG

In [0]:
latlong_df = province_latlong_df.withColumn('LAT_LONG', 
                                          fn.concat_ws(',', province_latlong_df['AVG_LAT'].cast(StringType()),
                                                       province_latlong_df['AVG_LONG'].cast(StringType()))).drop('AVG_LAT', 'AVG_LONG')
latlong_df.orderBy('CH_ID').show(truncate=False)                                                      

+-----+------------------------+------------------+-------------------------------------+
|CH_ID|CHANGWAT_E              |CHANGWAT_T        |LAT_LONG                             |
+-----+------------------------+------------------+-------------------------------------+
|10   |Bangkok                 |กรุงเทพมหานคร     |13.75614285714286,100.475            |
|11   |Samut Prakarn           |จ. สมุทรปราการ    |13.587,100.62938461538464            |
|12   |Nonthaburi              |จ. นนทบุรี        |13.850800000000003,100.48519999999999|
|13   |Pathum Thani            |จ. ปทุมธานี       |13.996428571428572,100.5409285714286 |
|14   |Phra Nakhon Si Ayutthaya|จ. พระนครศรีอยุธยา|14.355428571428574,100.56190476190474|
|15   |Ang Thong               |จ. อ่างทอง        |14.582999999999998,100.44899999999998|
|16   |Lop Buri                |จ. ลพบุรี         |14.780208333333333,100.62445833333334|
|17   |Sing Buri               |จ. สิงห์บุรี      |14.8995,100.40225000000001           |
|18   |Cha

In [0]:
latlong_df.count()

77

#Final

In [0]:
final_province_df = latlong_df.withColumn('CHANGWAT_T', fn.regexp_replace(latlong_df['CHANGWAT_T'], 'จ. ', ''))
final_province_df.show()

+-----+----------------+-----------+--------------------+
|CH_ID|      CHANGWAT_E| CHANGWAT_T|            LAT_LONG|
+-----+----------------+-----------+--------------------+
|   91|           Satun|       สตูล|6.617406249999999...|
|   31|        Buri Ram|  บุรีรัมย์|14.95968421052631...|
|   39|Nong Bua Lam Phu|หนองบัวลำภู|17.16793333333333...|
|   15|       Ang Thong|    อ่างทอง|14.58299999999999...|
|   82|       Phang-nga|      พังงา|8.469222222222223...|
|   38|       Bueng Kan|     บึงกาฬ|18.31841666666666...|
|   96|      Narathiwat|   นราธิวาส|6.387857142857142...|
|   75| Samut Songkhram|สมุทรสงคราม|13.40718181818181...|
|   93|     Phatthalung|     พัทลุง|7.606733333333334...|
|   57|      Chiang Rai|   เชียงราย|      19.8986,99.807|
|   74|    Samut Sakhon|  สมุทรสาคร|13.53783333333333...|
|   21|          Rayong|      ระยอง|12.67957894736841...|
|   61|     Uthai Thani|  อุทัยธานี|15.40414285714285...|
|   12|      Nonthaburi|    นนทบุรี|13.85080000000000...|
|   20|       

In [0]:
ordered_province_df = final_province_df.orderBy('CH_ID')
ordered_province_df.show(100)

+-----+--------------------+---------------+--------------------+
|CH_ID|          CHANGWAT_E|     CHANGWAT_T|            LAT_LONG|
+-----+--------------------+---------------+--------------------+
|   10|             Bangkok|  กรุงเทพมหานคร|13.75614285714286...|
|   11|       Samut Prakarn|    สมุทรปราการ|13.587,100.629384...|
|   12|          Nonthaburi|        นนทบุรี|13.85080000000000...|
|   13|        Pathum Thani|       ปทุมธานี|13.99642857142857...|
|   14|Phra Nakhon Si Ay...|พระนครศรีอยุธยา|14.35542857142857...|
|   15|           Ang Thong|        อ่างทอง|14.58299999999999...|
|   16|            Lop Buri|         ลพบุรี|14.78020833333333...|
|   17|           Sing Buri|      สิงห์บุรี|14.8995,100.40225...|
|   18|            Chai Nat|         ชัยนาท|15.18855555555555...|
|   19|            Saraburi|        สระบุรี|14.51227272727272...|
|   20|           Chon Buri|         ชลบุรี|13.34883333333333...|
|   21|              Rayong|          ระยอง|12.67957894736841...|
|   22|   

# Load 

เก็บข้อมูลที่ transform เเล้วไว้ใน drive

In [0]:
ordered_province_df.coalesce(1).write.csv('/content/drive/My Drive/Project/Covid19/single_province_lat_long.csv', header=True)